# Integrating a generic, non-python tool in pyiron (Level B, recommended approach)
To integrate a new tool into pyiron, one needs to create a pyiron job class, which is inherited from pyiron generic job class: `GenericJob`   

In [2]:
from os.path import join

In [3]:
from pyiron_base import GenericJob, GenericParameters

## Definition of the job class
Here, as a simple and generic process, we would like to use the bash command `cat` to print some values in an output file.    

In [4]:
class ToyBashJob(GenericJob):
    def __init__(self, project, job_name):
        super().__init__(project, job_name) 
        self.input = GenericParameters(table_name="input")
        self.input['input_energy'] = 100
        self.executable = "cat input > output"

    def write_input(self): 
        self.input.write_file( 
            file_name="input",
            cwd=self.working_directory
        )

    def collect_output(self):
        file = join(self.working_directory, "output") 
        with open(file) as f:
            line = f.readlines()[0]
        energy = float(line.split()[1]) 
        with self.project_hdf5.open("output/generic") as h5out: 
            h5out["energy_tot"] = energy

    def to_hdf(self, hdf=None, group_name=None): 
        super().to_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.to_hdf(h5in)

    def from_hdf(self, hdf=None, group_name=None): 
        super().from_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.from_hdf(h5in)

## Exercise 1:  
using the above job class, create a project and run a job of type `ToyBashJob` 

In [6]:
from pyiron_base import Project
pr = Project('test')
pr.remove_jobs_silently(recursive=True)
job = pr.create_job(job_type=ToyBashJob, job_name="toy")
print(job.input)
job.run()
job['output/generic/energy_tot']

      Parameter  Value Comment
0  input_energy    100        
The job toy was saved and received the ID: 64


/home/muhammad/miniconda3/envs/workshop_test/lib/python3.8/site-packages/pyiron_base/generic/hdfio.py:361: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  with h5py.File(self.file_name, mode="a", libver="latest", swmr=True) as h:


100.0